In [8]:
import cadquery as cq
from jupyter_cadquery.cadquery import (PartGroup, Part, Edges, Faces, Vertices, show, 
                                       replay, enable_replay, disable_replay, reset_replay)
from jupyter_cadquery import set_sidecar, set_defaults

show_object = show

In [9]:
set_sidecar("Gears")

In [10]:
enable_replay()
show_object = replay


Enabling jupyter_cadquery replay


In [11]:
set_defaults(axes=True, grid=True, axes0=True, edge_accuracy=0.001, timeit=True)

In [12]:
from math import pi, sin, cos, acos, tan, atan, ceil, radians, degrees, floor
import numpy as np
clearance = 0.05

In [13]:
def copier(solid, vect, number, distance, winkel):
    copies = []
    
    for i in range(number):
        copy = solid.rotate((0.0, 0.0, 0.0), (0.0, 0.0, 1.0), i * winkel)
        d = distance * i
        copy = copy.translate((vect[0] * d, vect[1] * d, vect[2] * d))
        copies.append(copy)
    
    return copies

def polar_to_cartesian(polvect):
    return (polvect[0] * cos(polvect[1]),
            polvect[0] * sin(polvect[1]))


#     Circle Involutes-Function:
#     Returns the Polar Coordinates of an Involute Circle
#     r = Radius of the Base Circle
#     rho = Rolling-angle in Degrees
def ev(r, rho):
    return (r / cos(rho), tan(rho) - rho)

def sphere_to_cartesian(vect):
    return (vect[0] * sin(vect[1]) * cos(vect[2]),  
            vect[0] * sin(vect[1]) * sin(vect[2]),
            vect[0] * cos(vect[1]))


In [14]:

def rack(modul, length, height, width, pressure_angle=20.0, helix_angle=0.0):
    # Dimension Calculations
    pressure_angle = radians(pressure_angle)
    helix_angle = radians(helix_angle)
    
    modul = modul * (1.0 - clearance)
    c = modul / 6.0                                              # Tip Clearance
    mx = modul / cos(helix_angle)                                # Module Shift by Helix Angle in the X-Direction
    a = 2.0 * mx * tan(pressure_angle) + c * tan(pressure_angle) # Flank Width
    b = pi * mx / 2.0 - 2.0 * mx * tan(pressure_angle)           # Tip Width
    x = width * tan(helix_angle)                                 # Topside Shift by Helix Angle in the X-Direction
    nz = ceil((length + abs(2.0 * x)) / (pi * mx))               # Number of Teeth
    
    
    underside_points = [[0, -c],
                        [a, 2.0 * modul],
                        [a + b, 2.0 * modul],
                        [2.0 * a + b, -c],
                        [pi * mx, -c],
                        [pi * mx, modul - height],
                        [0.0, modul - height]]

    topside_points = [[0.0 + x, -c],
                      [a + x, 2.0 * modul],
                      [a + b + x, 2.0 * modul],
                      [2.0 * a + b + x, -c],
                      [pi * mx + x, -c],
                      [pi * mx + x, modul - height],
                      [0.0 + x, modul - height]]
    
    res = cq.Workplane('XY')
    res = res.polyline(underside_points).close()
    res = res.workplane(offset=width)
    res = res.polyline(topside_points).close()
    res = res.loft()
    
    teeth = copier(res, (1.0, 0.0, 0.0), nz, pi * mx, 0.0)
    res = teeth[0]
    for t in teeth[1:]:
        res = res.union(t)
        
    box = cq.Workplane().box(length, height + modul + 1.0, width + 1.0, centered=False)
    box = box.translate((abs(x), -height + modul - 0.5, -0.5))
    res = res.intersect(box)
    res = res.translate([-pi * mx * (nz - 1.0) / 2.0 - a - b / 2.0, -modul, 0.0])

    return res

show_object(rack(modul=1.0, length=60.0, height=5.0, width=20.0, pressure_angle=20.0, helix_angle=0.0))

Use the multi select box below to select one or more steps you want to examine


In [ ]:

def mountable_rack(modul, length, height, width, pressure_angle, helix_angle, fasteners, profile, head):
    a_rack = rack(modul, length, height, width, pressure_angle, helix_angle)
    offset = length / fasteners;

    def ph_screw(idx):
        screw = cq.Workplane('XZ')
        screw = screw.circle(profile).extrude(profile * 0.6 + modul * 2.25)
        screw = screw.faces('>Y').circle(profile / 2.0).extrude(height + modul)
        screw = screw.translate((idx * offset + (offset - length) / 2.0, modul, width / 2.0))
        return screw
    
    def cs_screw(idx):
        screw = cq.Workplane('XZ')
        screw = screw.circle(profile * 1.5 / 2.0).extrude(profile * 1.25 + modul * 2.25)
        screw = screw.faces('>Y').circle(profile / 2.0).extrude(height + modul)
        screw = screw.translate((idx * offset + (offset - length) / 2.0, modul, width / 2.0))
        return screw
    
    def c_screw(idx):
        screw = cq.Workplane('XZ').workplane(modul * 2.25)
        screw = screw.circle(profile).workplane(offset=profile / 2.0)
        screw = screw.circle(profile / 2.0).loft()
        screw = screw.faces('>Y').circle(profile).extrude(-modul * 2.25)
        screw = screw.faces('<Y').circle(profile / 2.0).extrude(height + modul)
        screw = screw.translate((idx * offset + (offset - length) / 2.0, modul, width / 2.0))
        return screw
    
    def rc_screw(idx):
        screw = cq.Workplane('XZ').workplane(modul * 2.25 + profile / 4.0)
        screw = screw.circle(profile).workplane(offset=profile / 2.0)
        screw = screw.circle(profile / 2.0).loft()
        screw = screw.faces('>Y').circle(profile).extrude(-modul * 2.25 - profile / 4.0)
        screw = screw.faces('<Y').circle(profile / 2.0).extrude(height + modul)
        screw = screw.translate((idx * offset + (offset - length) / 2.0, modul, width / 2.0))
        return screw
    
    def css_screw(idx):
        screw = cq.Workplane('XZ').workplane(modul * 2.25)
        screw = screw.circle(profile).workplane(offset=profile * 0.6)
        screw = screw.circle(profile / 2.0).loft()
        screw = screw.faces('>Y').circle(profile).extrude(-modul * 2.25)
        screw = screw.faces('<Y').circle(profile / 2.0).extrude(height + modul)
        screw = screw.translate((idx * offset + (offset - length) / 2.0, modul, width / 2.0))
        return screw
    
    screw_f = {
        'PH': ph_screw,
        'CS': cs_screw,
        'C': c_screw,
        'RC': rc_screw,
        'CSS': css_screw,
    }[head]
    
    for i in range(fasteners):
        a_rack = a_rack.cut(screw_f(i))
    
    return a_rack

show_object(mountable_rack(modul=1.0, length=60.0, height=10.0, width=20.0,
                           pressure_angle=20.0, helix_angle=20.0, profile=3.0, head='CSS',fasteners=3))

In [15]:
def spur_gear(modul, tooth_number, width, bore, pressure_angle=20.0, helix_angle=0.0, optimized=True):
    # Dimension Calculations
    pressure_angle = radians(pressure_angle)
    helix_angle = radians(helix_angle)
    
    d = modul * tooth_number                                   # Pitch Circle Diameter
    r = d / 2                                                  # Pitch Circle Radius
    alpha_spur = atan(tan(pressure_angle) / cos(helix_angle))  # Helix Angle in Transverse Section
    db = d * cos(alpha_spur)                                   # Base Circle Diameter
    rb = db / 2                                                # Base Circle Radius
    da = (d + modul * 2.2) if modul < 1.0 else d + modul * 2.0 # Tip Diameter according to DIN 58400 or DIN 867
    ra = da / 2.0                                              # Tip Circle Radius
    c = 0.0 if tooth_number < 3 else modul / 6.0               # Tip Clearance
    df = d - 2.0 * (modul + c)                                 # Root Circle Diameter
    rf = df / 2.0                                              # Root Radius
    rho_ra = acos(rb / ra)                                     # Maximum Rolling Angle;
                                                               # Involute begins on the Base Circle and ends at the Tip Circle
    rho_r = acos(rb / r)                                       # Rolling Angle at Pitch Circle;
                                                               # Involute begins on the Base Circle and ends at the Tip Circle
    phi_r = tan(rho_r) - rho_r                                 # Angle to Point of Involute on Pitch Circle
    gamma = width / (r * tan(pi / 2.0 - helix_angle))          # Torsion Angle for Extrusion
    step = rho_ra / 16                                         # Involute is divided into 16 pieces
    tau = pi * 2.0 / tooth_number                              # Pitch Angle
    
    r_hole = (2.0 * rf - bore) / 8.0                           # Radius of Holes for Material-/Weight-Saving
    rm = bore / 2.0 + 2.0 * r_hole                             # Distance of the Axes of the Holes from the Main Axis
    z_hole = floor(2 * pi * rm / (3.0 * r_hole))               # Number of Holes for Material-/Weight-Saving
    
    optimized = (optimized and r >= width * 1.5 and d > 2.0 * bore)   # is Optimization useful?
    tooth_width = (pi * (1.0 - clearance)) / tooth_number + 2.0 * phi_r
    
    def tooth_curve(t, left):
        invc = tan(t) - t
        rad_ = rb / cos(t)
        if left:
            return (rad_ * cos(invc),
                    rad_ * sin(tooth_width - invc))
        else:
            return (rad_ * cos(invc),
                    rad_ * sin(invc))

    tooth = (
        cq.Workplane('XY').
        # Right side
        parametricCurve(lambda t: tooth_curve(t, False), 16, 0.0, rho_ra).
        # Tip of the tooth
        moveTo(*tooth_curve(rho_ra, False)).lineTo(*tooth_curve(rho_ra, True)).
        # Left side
        parametricCurve(lambda t: tooth_curve(t, True), 16, 0.0, rho_ra).
        # Root
        moveTo(*tooth_curve(0.0, True)).lineTo(0.0, 0.0).
        lineTo(*tooth_curve(0.0, False)).
        consolidateWires()
    )
        
    tooth = tooth.twistExtrude(width, degrees(gamma))
    res = cq.Workplane('XY').circle(rf).extrude(width).union(tooth)

    for rot in np.arange(tau, pi * 2.0, tau):
        res = res.union(tooth.rotate((0.0, 0.0, 0.0), (0.0, 0.0, 1.0), degrees(rot)))
        
    # "bore"
    res = res.faces('<Z').circle(rm + r_hole * 1.49).cutThruAll()
        
    body = cq.Workplane('XY')
    
    if optimized:
        res = (res.faces('<Z').
               circle((bore + r_hole) / 2.0).
               circle(bore/2).
               extrude(width))
        
        if (width - r_hole / 2.0) < (width * 2.0 / 3.0):
            ht = width * 2.0 / 3.0
        else:
            ht = width - r_hole / 2.0
    
        body = body.circle(rm + r_hole * 1.51)
        body = body.circle((bore + r_hole) / 2.0)
        for i in range(z_hole):
            pos = sphere_to_cartesian((rm, pi / 2.0, i * pi * 2.0 / z_hole))
            body = body.moveTo(pos[0], pos[1]).circle(r_hole)
        body = body.consolidateWires().extrude(ht)
    else:
        body = body.circle(rm + r_hole * 1.51)
        body = body.circle(bore / 2.0)
        body = body.consolidateWires().extrude(width)

    res = res.union(body)
    return res
    
gear = spur_gear(modul=1.0, tooth_number=42, width=5.0, bore=4.0, pressure_angle=20.0, helix_angle=20.0, optimized=True)
show(gear)


| | Object: Group
| | | (Caching 140392217899824)
| | | build mesh time:    0.72 sec
| | | discretize time:    0.02 sec
| | | edge list:        0.01 sec
| | shape render time:    0.76 sec
| overall render time:    0.77 sec
| create bounding box:    0.22 sec
| configure view:       0.03 sec
add shapes:             1.03 sec
configure display:      0.01 sec
Done, using side car 'Gears'


In [ ]:
def spur_gear_profile(module, teeth_number, pressure_angle=20.0):
    
    
    